In [2]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
import csv
import pandas as pd
import json
import time
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
import string 
import unicodedata as ud
from greek_stemmer import GreekStemmer
import pymongo
import numpy as np
import re
import networkx as nx
import indexer as ind
mongo_client = pymongo.MongoClient("mongodb://localhost:27017/")
#mongo_client.drop_database("GreekParliamentProceedings")
client = mongo_client["GreekParliamentProceedings"]
index = client["InvertedIndex"]
database = client["Database"]

In [12]:
document = list(database.find({"_id":"1"}, { "_id": 0, "speech": 1 }))
speech = document[0]['speech']

In [18]:
import spacy
nlp = spacy.load('el_core_news_md')
doc = nlp(speech)
for sent in doc.sents:
    print(sent)

 
Παρακαλείται ο κύριος Γραμματέας να συνοδεύσει την Ιερά Σύνοδο εκτός της Αιθούσης της Βουλής.
 
.
Παρακαλείται ο συνάδελφος Βουλευτής κ. Σαδίκ Αμέτ, που ανήκει στο Μωαμεθανικό Θρήσκευμα να προσέλθει και να δώσει τον οριζόμενο από το Σύνταγμα όρκο επί του Κορανίου.
 :
~"Ορκίζομαι στο όνομα του Παντοδύναμου Θεού και του μόνου αυτού Προφήτη ο οποίος είναι ο Μωάμεθ να είμαι πιστός στην πατρίδα και το δημοκρατικό πολίτευμα, να υπακούω στο Σύνταγμα και τους νόμους και να εκπληρώνω ευσυνείδητα τα καθήκοντά μου".


In [20]:
import syntok.segmenter as segmenter

# choose the segmentation function you need/prefer
for sentence in doc.sents:
    for txt in segmenter.process(sentence):
        print(txt)
    print('\n')

TypeError: expected string or buffer

In [17]:
def keyword_extraction(text, w, n, theta, language):
    #STEP 1: SPLIT TEXT INTO SENTENCES AND PRE-PROCESS 
    nlp = spacy.load('el_core_news_md')
    sentences = nlp(speech)
    for sentence in sentences.sents:
        #SPLIT SENTENCE INTO CHUNKS
        import segtok.segmenter as sn
        chunks = sn.